In [1]:
from gurobipy import GRB, Model, quicksum
import pandas as pd

## Daten

Erzeuge Dataframes für Eingangsdaten

In [2]:
df = pd.read_csv('edges.csv', header=None, names=['Start', 'Ziel', 'Kosten'])

In [3]:
df_cap = pd.read_csv('capacities.csv', header=None, names=['Knoten', 'Kapazität'])

Aus den Dataframes werden nun Mengen, Listen und Dictionaries erstellt, welche die Eingangsdaten des Optimierungsmodells enthalten

In [4]:
df_cap.values

array([['T1', 10],
       ['T2', 5],
       ['T3', 25]], dtype=object)

In [5]:
capacities = {
    n: cap for (n, cap) in df_cap.values
}

In [6]:
edges = set([tuple(x) for x in df[['Start', 'Ziel']].values])

In [7]:
costs = {
    (n, n1): w for (n, n1, w) in df.values
}

In [8]:
nodes = set(df['Start'].append(df['Ziel']))

In [9]:
nodes_intermed = [n for n in nodes if n[0] == 'T']

## Optimierungsmodell

Modelldeklaration

In [10]:
m = Model('Fluss')

Using license file c:\gurobi\gurobi.lic
Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value 427039-DSM
Compute Server job ID: 1f6ca1fa-bfa2-4894-b1e4-59d540db64ea
Capacity available on '427039-DSM' cloud pool - connecting...
Established HTTPS encrypted connection


Entscheidungsvariablen

In [11]:
x = m.addVars(edges, name='Transport auf Kante', lb=0)

30 Einheiten werden zur Senke hin und von der Quelle weg transportiert

In [12]:
m.addConstr(
    quicksum(x[n, 'S'] for n in nodes if (n, 'S') in edges) == 30
)

m.addConstr(
    quicksum(x['Q', n] for n in nodes if ('Q', n) in edges) == 30
)

m.update()

Kapazitätsrestriktionen

In [13]:
cap_restr = m.addConstrs(
    quicksum(x[n1, n] for n1 in nodes if (n1, n) in edges) <= capacities[n] for n in nodes_intermed
)
m.update()

Flusserhaltungsgleichung

In [14]:
m.addConstrs(
    quicksum(x[n1, n] for n1 in nodes if (n1, n) in edges) == quicksum(x[n, n2] for n2 in nodes if (n, n2) in edges)
    for n in nodes_intermed
)
m.update()

Zielfunktion

In [15]:
m.setObjective(
    quicksum(costs[e] * x[e] for e in edges), sense=GRB.MINIMIZE
)

In [16]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 8 rows, 8 columns and 21 nonzeros
Model fingerprint: 0x2020b2f0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 3e+01]
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  4.600000000e+02


In [17]:
for e in x:
    if x[e].X > 0:
        print(f"Kante: {e}, Transportmenge: {x[e].X}")

Kante: ('Q', 'T1'), Transportmenge: 10.0
Kante: ('T3', 'S'), Transportmenge: 20.0
Kante: ('Q', 'T3'), Transportmenge: 15.0
Kante: ('Q', 'T2'), Transportmenge: 5.0
Kante: ('T2', 'T3'), Transportmenge: 5.0
Kante: ('T1', 'S'), Transportmenge: 10.0


## Sensitivitätsanalyse

Schattenpreis

In [18]:
cap_restr['T3'].Pi

0.0

RHS der Kapazitätsrestriktion

In [19]:
cap_restr['T1'].rhs

10.0

Größte RHS für die der Schattenpreis noch gültig ist.

In [20]:
cap_restr['T1'].SARHSUp

25.0

Gültigkeitsbereich des Schattenpreises

In [21]:
cap_restr['T1'].SARHSUp - cap_restr['T1'].rhs

15.0

Reduced Costs

In [22]:
for k in x:
    print(x[k].rc)

0.0
0.0
0.0
5.0
17.0
0.0
0.0
0.0
